In [60]:
import os
from pydantic import Extra
import requests
from typing import Any, List, Mapping, Optional

from langchain.callbacks.manager import CallbackManagerForLLMRun
from langchain.llms.base import LLM
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

from langchain.prompts import PromptTemplate
# Run chain
from langchain.chains import RetrievalQA
from langchain_community.vectorstores import FAISS # type: ignore
from langchain_ollama import OllamaEmbeddings # type: ignore
import warnings
warnings.filterwarnings('ignore')

In [61]:
os.environ["token"] = "Bearer token|123675a6-95f6-4fb7-bc95-30095472ae3a|02de311fd83421a7fd637bf34dc8f959caa29f39888d2919e4b9640a2220224b"
token = os.environ["token"]

In [62]:
skill1="Core Java L1"
skill2="Spring Boot L2"
skill3="Hibernate L3"
skill4="Selenium L4"

In [63]:
from typing import ClassVar

parser = StrOutputParser()

class LlamaLLM(LLM):
    llm_url: ClassVar[str] = 'https://api.lab45.ai/v1.1/skills/completion/query'
    
    backend:        Optional[str]   = 'gpt-35-turbo-16k'
    temp:           Optional[float] = 0.7
    top_p:          Optional[float] = 0.1
    top_k:          Optional[int]   = 40
    n_batch:        Optional[int]   = 8
    n_threads:      Optional[int]   = 4
    n_predict:      Optional[int]   = 256
    max_tokens:     Optional[int]   = 256
    repeat_last_n:  Optional[int]   = 64
    repeat_penalty: Optional[float] = 1.18

    class Config:
        extra = Extra.forbid

    @property
    def _llm_type(self) -> str:
        return "gpt-35-turbo-16k"
    
    @property
    def _get_model_default_parameters(self):
        return {
            "max_tokens": self.max_tokens,
            #"n_predict": self.n_predict,
            "top_k": self.top_k,
            "top_p": self.top_p,
            "temperature": self.temp,
            #"n_batch": self.n_batch,
            #"repeat_penalty": self.repeat_penalty,
            #"repeat_last_n": self.repeat_last_n,
        }

    def _call(
        self,
        skill1: str,
        user: str,
        stop: Optional[List[str]] = None,
        run_manager: Optional[CallbackManagerForLLMRun] = None,
        **kwargs: Any,
    ) -> str:
        if stop is not None:
            raise ValueError("stop kwargs are not permitted.")

        payload = {
        "messages": [
            {
            "content": "As a skilled Technical Interview Panel with advanced knowledge in technology and data science, your role is to meticulously evaluate a employer job description and provide detailed job description having below points outlined.",
            "role": "system",
            "content": "Use the mentioned skills to generate the job description, where L1 means Beginner, L2 means Intermediate, L3 means Advanced, and L4 means Expert.",
            "role": "user",
            "content": """1. **Job Title and Summary**:
            - "What are the key responsibilities?"
            
            2. **Key Responsibilities**:
            - "List the main duties and responsibilities."
            - "What are the daily tasks involved?"

            3. **Qualifications and Skills**:
            - "What qualifications are required?"
            - "List the essential skills needed."

            4. **Experience Requirements**:
            - "How many years of experience are needed?"
            - "What type of previous experience is beneficial?"

            """,
            "role": "assistant",
            "content": skill1,
            "role": user
            }
        ],
        "skill_parameters": {
            "model_name": "gpt-35-turbo-16k",
            "max_output_tokens": 4096,
            "temperature": 0,
            "top_k": 5
        },
        "stream_response": False
        }

        headers = {"Content-Type": "application/json","Authorization": token}

        response = requests.post(self.llm_url, json=payload, headers=headers, verify=False)
        response.raise_for_status()

        # print("API Response:", response.json())

        return response.json()  # get the response from the API

    @property
    def _identifying_params(self) -> Mapping[str, Any]:
        """Get the identifying parameters."""
        return {
            "llmUrl": self.llm_url,
            'model_parameters': self._get_model_default_parameters
            }

In [64]:
llm = LlamaLLM()

In [65]:
jd="Good in C++ with 5 years of Experience"

In [66]:
input_prompt = """

As a skilled Technical Interview Panel with advanced knowledge in technology and data science, your role is to meticulously evaluate a employer job description and provide detailed job description having below points outlined. 
Your evaluation will involve analyzing the job description for relevant skills, experiences that align with the job requirements. Look for key buzzwords and specific criteria outlined in the job description to determine the detailed job description
Your evaluation should be thorough, precise, and objective, ensuring that the most relevant job description is generated based on Skills and Experience mentioned
Your evaluation of job description should attract the right candidates"

1. **Job Title and Summary**:
   - "What are the key responsibilities?"
   
2. **Key Responsibilities**:
   - "List the main duties and responsibilities."
   - "What are the daily tasks involved?"

3. **Qualifications and Skills**:
   - "What qualifications are required?"
   - "List the essential skills needed."

4. **Experience Requirements**:
   - "How many years of experience are needed?"
   - "What type of previous experience is beneficial?"

"""

In [69]:
#Testing
#prompt = "[INST] Question: Who is Albert Einstein? \n Answer: [/INST]"
prompt = "Generate Job Description for Good in C++ with 5 years of Experience"
user = "user"
result = llm._call(skill4,user)
parsed_result = result['data']['content'] # type: ignore
parser.invoke(parsed_result)


"It seems like you're referring to the Selenium Level 4 (L4) autonomous driving system. The L4 designation indicates a high level of automation, where the vehicle can operate without human intervention in certain conditions or environments. This level of automation is just one step below full autonomy (L5), where the vehicle can operate in all conditions without human intervention.\n\nSelenium L4 systems are designed to handle most driving situations, but they may still require human intervention in certain scenarios, such as extreme weather conditions or unusual road layouts. These systems typically rely on a combination of sensors, cameras, radar, and advanced algorithms to navigate and make driving decisions.\n\nIf you have specific questions about Selenium L4 or autonomous driving systems in general, feel free to ask!"

In [ ]:
prompt = ChatPromptTemplate.from_messages(
    [("system", "you are a bot {name}"), ("human", "{input}")]
)
chain = prompt | llm



In [ ]:

# Build prompt

template = """[INST] <<SYS>>

Answer the question base on the context below.

<</SYS>>

Context: {context}
Question: {question}
Answer:
[/INST]"""
QA_CHAIN_PROMPT = PromptTemplate(input_variables=["context", "question"],template=template,)


qa_chain = RetrievalQA.from_chain_type(llm,
                                       verbose=True,
                                       retriever=None,
                                       #retriever=custom_retriever,
                                       return_source_documents=True,
                                       chain_type_kwargs={"prompt": QA_CHAIN_PROMPT})

In [ ]:
question = "Is probability a class topic?"
result = qa_chain({"query": question})
result["result"]
